# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, nltk
pd.set_option('max_colwidth', 100, 'display.max_rows', 10)

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## **Build an Automated Gender Classifier**

<span style="color:black">In this notebook, you will build an automated gender classifier trained on a corpus of names that is stored in two files, one for male names and one for female names. When training classifiers, it is important to use a balanced dataset. Download the dataset and balance the two labels by randomly subsampling the larger group. The size of the two should be equal and around 3000.

In [ ]:
_ = nltk.download(['names'], quiet=True)
LsM = nltk.corpus.names.words('male.txt')
LsF = nltk.corpus.names.words('female.txt')

# Balance observations in two classes. So, a random draw has 50% chance of either class.
np.random.seed(0)
LsF = sorted(list(np.random.choice(LsF, size=len(LsM), replace=False))) 
print(f'{len(LsM)} male names:  ', LsM[:20])
print(f'{len(LsF)} female names:', LsF[:20])

<span style="color:black">You can check whether some names from different cultures are included in this dataset.

In [ ]:
print('Oleg' in LsM, 'Olga' in LsF)
print('Ajay' in LsM, 'Asha' in LsF)
print('Chen' in LsM, 'Jiao' in LsF)

<span style="color:black">Recognize that the list of names is not comprehensive and some common names are missing. This is okay, because you are interested in building a model that can classify a name into gender categories based on lower level attributes, not the names themselves.

## <span style="color:black">Creating the Dataset</span>

<span style="color:black">It is prohibitive and inefficient to derive and maintain the full list of names. Instead, you want to extract the name attributes (i.e., features) that are indicative of a gender category for your model. Features are typically numeric because most models have numeric inputs and outputs, and selecting which features to use is an important part of the model selection process. Here, you will use character length of the name as a feature.

<span style="color:black">Begin by concatenating the two lists with their labels and adding a character length attribute as a column in the new dataframe.

In [ ]:
pd.set_option('max_colwidth', 100, 'display.max_rows', 10)
df0 = pd.DataFrame(dict(Name=LsF + LsM, Y=[1]*len(LsF) + [0]*len(LsM)) )
df0['Len'] = df0.Name.apply(len)
df0 = df0.set_index('Name')
df0.T

<span style="color:black">Next, split the observations into training and validation samples for training the model and evaluating the model's performance, respectively.

In [ ]:
from sklearn.model_selection import train_test_split
tX, vX, tY, vY = train_test_split(df0.drop('Y', axis=1), df0.Y, test_size=0.2, random_state=0)
print('tX:', tX.shape, ',\t tY:', tY.shape)
print('vX:', vX.shape, ',\t vY:', vY.shape)

## **Logistic Regression Model**

<span style="color:black"> The baseline model that you will use is the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model, which can be loaded from the `sklearn` package. To effectively utilize this model, three steps are required:

1. <span style="color:black">Instantiate a model object `lr` with default or specific parameters.
1. <span style="color:black">Fit the model object `lr` on the training inputs and outputs using the `fit()` method. In this step, the model learns the associations between inputs and outputs.
1. <span style="color:black">Evaluate the performance of the fitted (or trained) model on the validation set using the `score()` method. If you are not satisfied with the model's performance, you can continue modifying features or hyperparameters. If you are pleased with the model's performance, then you can place it into production and use the `predict()` method to predict the gender of the name argument.

<span style="color:black"> <b>Note:</b> You are encouraged to investigate the documentation of the object, its fit method, and its scoring method.

In [ ]:
from sklearn.linear_model import LogisticRegression
# ?LogisticRegression
# ?LogisticRegression.fit
# ?LogisticRegression.score

<span style="color:black"> Now, execute all three steps above: instantiate the model with the default arguments, fit it on the training set, and evaluate the trained model performance on the validation set. 

In [ ]:
lr = LogisticRegression()
lr.fit(tX, tY)
print(f'Accuracy = fraction of correct predictions: {lr.score(vX, vY):.3f}')

<span style="color:black">The resulting accuracy is 50.3%. Without standard deviation, it is hard to tell whether this accuracy has any significance, but given that the validation set has 1178 observations, even 0.3% difference from 50/50 classification seems to be significant. 

## **Make Predictions With a Trained Model**

<span style="color:black">There are two methods to display the predictions from the model. One way is to show the probability of the male label and probability of the female label. The `predict_proba()` method, which takes the feature value (length 4 for the names `'Oleg'` or `'Olga'`), and outputs these probabilities. Since males are represented by a 0 class, they are in the first position of the resulting array. So, 
    
$$\begin{align}
\mathbb{P}[\text{gender}&=\text{male or 0}|\text{name length}=4]=0.549\\
\mathbb{P}[\text{gender}&=\text{female or 1}|\text{name length}=4]=0.451
\end{align}$$

<span style="color:black">The second way is to show the categorical decision, which is simply the gender with the largest probability. In this case, this decision can be made by thresholding the probabilities at 50% and selecting the gender that exceeds 50%.

In [ ]:
# predicts a female for the four-letter name, like Oleg, Olga, 
print(f'Prediction probabilities for classes 0 (male) & 1 (female): ', lr.predict_proba([[4]]).round(3))
print(f'Predicted class based on (default) 0.5 threshold:', lr.predict([[4]]))

<span style="color:black">In summary, the trained model classifies any four-letter name as male gender. Although this is suboptimal, but you just trained the model to do non-random decisions at scale. This model can now classify millions of names in seconds. The large error can still be improved as you find better features and hyperparameters for the model.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now, equipped with the concepts and tools, tackle a few related tasks.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## Task 1

Use the model above to compute the probability of the name `'Elizabeth'` to be female. What is the predicted gender category for the 50% threshold?

<b>Hint:</b> You cannot feed the string into the model because it was trained on a single numeric feature, length of the name. First you must convert the name to its length and then use prediction methods to estimate probabilities of class male (identified by 0) and of class female (identified by 1). 


In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
x = len('Elizabeth')
print(f'Length of name={x}')
print(f'Prediction probabilities for classes 0 (male) & 1 (female): ', lr.predict_proba([[x]]).round(3))
print(f'Predicted class based on (default) 0.5 threshold:', lr.predict([[x]]))</pre>

In this example the model is correctly predicting the name to be of female gender with a 0.576 probability.
    </details> 
</font>
<hr>

## Task 2

Use the trained model above to compute the probability of the name `'Al'` to be male. What is the predicted gender category for the 50% threshold?

<b>Hint:</b> Refer to the the task above.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
x = len('Al')
print(f'Length of name={x}')
print(f'Prediction probabilities for classes 0 (male) & 1 (female): ', lr.predict_proba([[x]]).round(3))
print(f'Predicted class based on (default) 0.5 threshold:', lr.predict([[x]]))</pre>

Here the model is also correctly predicting a male category with 59.7% probability.
</details>
</font>
<hr>

## Task 3

Compute `P[Y=0|Len]` and `P[Y=1|Len]` for each possible value of `Len`. What is the length for the highest probability of any class? 

<b>Hint:</b> Apply the <code>predict_proba</code> to each value of <code>Len</code>. This can be done in a loop via list comprehension. The results can be packaged as a dataframe for convenience and coloring.


In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
LLnAllProb = [([x] + lr.predict_proba([[x]]).flatten().tolist()) for x in range(df0.Len.min(),df0.Len.max()+1)]
dfAllProb = pd.DataFrame(LLnAllProb, columns=['Len','P[Y=male or 0|Len]','P[Y=female or 1|Len]']).set_index('Len')
dfAllProb.T.style.background_gradient(axis=1).set_precision(3)</pre>

Notice that the probabilities are complementary. I.e. `P[Y=0|Len] + P[Y=1|Len]=1`. The largest probability is `P[Y=1|Len=15]=0.712`. In fact we observe a bias for longer names to be feminine and shorter names to be masculine.
    </details> 
</font>
<hr>

## Task 4

For each name length, compute the fraction of the count of male names to the count of female names. If you were to use these ratios to classify `'Al'` and `'Elizabeth'`, what predictions would you make? How does it compare to predictions from logistic regression above?

<b>Hint:</b> Try a <code>pivot</code> or <code>pivot_table()</code> method of dataframe. Alternatively, you can use loops to compute these counts or even compute it with the <code>groupby()</code> method of a dataframe for each Y label separately and then merge the resulting dataframes.


In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
# df0.append([1]).fillna(1).rename(columns={0:'count'})  # alternative way of adding a column of ones
df0_ = df0.copy()   # create a copy of a dataframe
df0_['count'] = 1   # add a column of ones
df0Pvt = df0_.pivot_table(index='Len', columns='Y', values='count', aggfunc='count') # aggregate by Y and by Len, calc counts
df0Pvt = df0Pvt.rename(columns={0:'male', 1:'female'})
df0Pvt['#male/#female'] = df0Pvt.male / df0Pvt.female
df0Pvt.style.background_gradient(axis=0)</pre>

We would strongly predict a male gender for the two-letter name because there are 3.6x more male names. Also, we would still predict a nine-letter name to be of a female gender because there are only 0.89x male names with nine letters. Notice that the logistic regression above strongly recommends a fifteen-letter name to be in the female category, but the ratio below is indifferent, since we only have one fifteen-letter name from each category. However, we do not know whether this name made it to the training set or not. It is likely that logistic regression is performing better with shorter lengths for which we have more observations.   
    </details> 
</font>
<hr>

## Task 5

Retrain the model with `train_test_split` seeded with value 1 (instead of a 0 as you did above). Then refit the model to the resulting training set and score the model on the validation set. How does the resulting accuracy compare to that above?

<b>Hint:</b> Repeat the train/test split, training and scoring from above.


In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
tX, vX, tY, vY = train_test_split(df0.drop('Y', axis=1), df0.Y, test_size=0.2, random_state=1)
print(f'Accuracy = fraction of correct predictions: {LogisticRegression().fit(tX, tY).score(vX, vY):.3f}')</pre>

The accuracy is now 0.532. It is more discriminative than the one above, but we did nothing special except for picking a different training set. Notably, our model's accuracy is a function of the training set. A different training set is likely to produce a slightly different accuracy and predictions.
    </details> 
</font>
<hr>